In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")

from agents.tester import Tester

from utils.config import *

# JSON = 'configs/pre_trained/ResNet18.json'
# JSON = 'configs/pre_trained/ResNet34.json'
# JSON = 'configs/pre_trained/ResNetSE34L.json'
# JSON = 'configs/pre_trained/ResNetSE34V2.json'
JSON = 'configs/pre_trained/AutoSpeech.json'
# JSON = 'configs/pre_trained/Cnn3D.json'
# JSON = 'configs/pre_trained/SincNet.json'
# JSON = 'configs/pre_trained/MobileNet.json'

config = process_config(JSON)

agent = Tester(config)
agent.run()

[INFO]: Experiment: Cnn3D Pretrained.
